As part of the py3-ification project, we're writing lots of tests. Check out all the __*_test.txt*__ files in the repo to see some of them!

These are some random lessons I've learned while working on those tests.

Mimicking User Input
===========

How can we test a function that takes in data with ``raw_input``?

In [ ]:
"""Run me to see the usual way input works!"""

for i in range(3):
    fruit = input("Hey, give me some fruit! > ")    # Python 3 uses input, not raw_input
    print(f"Yay, I got {fruit}!")                   # Python 3.6 f string!

This appears in, for example, the Guessing Game exercise. After the user executes the
Python file contaning the game, they're prompted for input:

    Howdy, what's your name?
    (type in your name) Jessica
    Jessica, I'm thinking of a number between 1 and 100.
    Try to guess my number.
    Your guess? 50
    Your guess is too low, try again.
    Your guess? 80
    Your guess is too high, try again.
    Your guess? 60
    Your guess is too low, try again.
    Your guess? 70
    Your guess is too high, try again.
    Your guess? 63
    Your guess is too low, try again.
    Your guess? 64
    Your guess is too low, try again.
    Your guess? 67
    Your guess is too low, try again.
    Your guess? 69
    Your guess is too high, try again.
    Your guess? 68
    Well done, Jessica! You found my number in 9 tries!
    
And of course, they can keep guessing until they get the number right. 


In [1]:
import sys

In [7]:
def fake_input(prompt):
    """Display prompt to stdout & return first item from _inputs."""
    
    # Prompt becomes whatever you pass raw_input/input.
    
    # If you were to just print the prompt instead, you'd end up with
    # an extra space after it, so we write the prompt to stdout directly
    # instead. Try it, though. :)
    
    # print(prompt, end=" ")
    
    sys.stdout.write(prompt)
    
    v = _inputs.pop(0)
    
    print(v)
    return v

# The funnest part: replacing the builtin raw_input or input!

try:
    import __builtin__ as b    # Python 2
except ImportError:
    import builtins as b       # Python 3
    
b.raw_input = b.input = fake_input

# Make a list of items for fake_input to pop from
_inputs = ["apples", "berries", "cherries"]

# And ask for our inputs.
     
for i in range(3):
    fruit = input("Hey, give me some fruit! > ")    # Python 3 uses input, not raw_input
    print(f"Yay, I got {fruit}!")                   # Python 3.6 f string! <3

Hey, give me some fruit! > apples
Yay, I got apples!
Hey, give me some fruit! > berries
Yay, I got berries!
Hey, give me some fruit! > cherries
Yay, I got cherries!


Also experimented with StringIO, but didn't end up using it because the output didn't match the game's actual output, visually.

- reads from/writes to a string buffer. 
- behaves kind of like a file object

Docs for py2: https://docs.python.org/2/library/stringio.html

Docs for py3: https://docs.python.org/3/library/io.html#io.StringIO



Mocking Output That Can Change
================

Remember that time Mel wanted to implement Rush Hour pricing, where melons cost $4 more if the 
time is between 8 AM and 11 AM on a weekday? I do, and if you've done oo-melons, you probably do, too.

The Code to Test:
----------------

In [ ]:
def get_base_price(self):
    """Calculate base price using splurge pricing and rush hour fee."""

    # Splurge rate
    base_price = random.randrange(5, 10)

    now = datetime.datetime.now()

    # Is it rush hour?
    if now.hour >= 8 and now.hour <= 11 and now.weekday() < 5:
        base_price += 4

    return base_price

We never want to be testing someone else's code. Two problem points, then: `randrange` and `datetime.now()`. 

Overwriting `random.randrange()`
------------------------------

...was the easy part.



In [8]:
import random
random.randrange = lambda x, y: 42

In [9]:
random.randrange(10, 100)

42

Okay, but do you believe me?

In [10]:
random.randrange(1, 4)

42

Yay! It's really gone!

Overwriting `datetime.now()`
--------------------------

...was a little trickier, but still very doable. :)

Let's get the current time:

In [11]:
import datetime

datetime.datetime.now()

datetime.datetime(2017, 4, 11, 9, 40, 0, 254806)

Awesome. Funnily enough, when students hit this piece of the Further Study, it's usually not rush hour, so how
can they know their code works...?

And how can we test our solution code reliably at any time? First, I tried this:

In [12]:
fake_now = lambda: datetime.datetime(2017, 4, 3, 9, 0)
fake_now()

datetime.datetime(2017, 4, 3, 9, 0)

In [13]:
datetime.datetime.now = fake_now

TypeError: can't set attributes of built-in/extension type 'datetime.datetime'

Sadness! We're just going to have to nuke the whole class, then.

In [14]:
class RushHourDatetime(datetime.datetime):
    @classmethod
    def now(cls):
        return datetime.datetime(2017, 4, 3, 9)       
datetime.datetime = RushHourDatetime

In [15]:
datetime.datetime.now()

RushHourDatetime(2017, 4, 3, 9, 0)

In [19]:
def get_cool_num():
    """Return a cool number, based on the time of day."""
    
    now = datetime.datetime.now()
    
    if now.hour > 8 and now.hour < 11:
        print(f"The current datetime is: {now}")
        return "42...or maybe e...or maybe j...I can't make up my mind!"
    else:
        return 13
    
print(f"A cool number, you say? How about {get_cool_num()}")

The current datetime is: 2017-04-03 09:00:00
A cool number, you say? How about 42...or maybe e...or maybe j...I can't make up my mind!


Yay! But now what happens if we need to test that the prices come out correctly when it's not rush hour?

In [25]:
class NormalHourDatetime(datetime.datetime):
    """An ordinary time of day."""
    
    @classmethod
    def now(cls):
        return datetime.datetime(2017, 4, 3, 16)
    
datetime.datetime = NormalHourDatetime
datetime.datetime.now().hour

TypeError: object() takes no parameters

Oops, we don't seem to be able to get at __*datetime.datetime*__ that way.

We could just return a __*RushHourDatetime*__, which inherited from the original __*datetime.datetime*__ intsead of trying to use __*datetime.datetime*__:

In [27]:
class NormalHourDatetime():
    """An ordinary time of day."""
    
    @classmethod
    def now(cls):
        return RushHourDatetime(2017, 4, 3, 16)
    
datetime.datetime = NormalHourDatetime
datetime.datetime.now().hour

16

But feels wrong. Perhaps we could have just made a __*FakeDatetime*__ class instead... 

Ended up just making some datetime objects in advance, and switching 'em out.

**(You may have to reset the kernel here.)**

In [1]:
import datetime

NORMAL_HOUR = datetime.datetime(2017, 4, 3, 16)
RUSH_HOUR = datetime.datetime(2017, 4, 3, 9)

class NormalHourDatetime():
    """An ordinary time of day."""
    
    @classmethod
    def now(cls):
        return NORMAL_HOUR
    
class RushHourDatetime():
    """A time of day where melon prices are gloriously high."""
    
    @classmethod
    def now(cls):
        return RUSH_HOUR

In [2]:
print(f"Normal time: {NORMAL_HOUR}", f"Rush hour time: {RUSH_HOUR}", sep="\n")

Normal time: 2017-04-03 16:00:00
Rush hour time: 2017-04-03 09:00:00


Now, we can overwrite __*datetime.datetime*__ with either class, as needed.

In [3]:
datetime.datetime = NormalHourDatetime

print(f"Overwritten with normal: {datetime.datetime.now()}")

Overwritten with normal: 2017-04-03 16:00:00


In [4]:
datetime.datetime = RushHourDatetime

print(f"Overwritten with rush hour: {datetime.datetime.now()}")

Overwritten with rush hour: 2017-04-03 09:00:00


Yay!

(If anyone has ideas about a "best" way to approach this problem, I'd be very interested in discussing!)

So We're Adding Doctests to the RST, Too
======================

We're putting doctests directly into our lectures, exercises, etc.

It's really cool! Take a look at dicts-1 and classes lectures for examples.

There's even a fun new `make` command you can run: ``make doctest``.

The setup for these tests usually isn't too bad, but when you start involving databases and Flask, things get interesting.

Dropping DBs in RST (and Python, Generally)
----------------------------------------

The steps we usually teach students for testing database code (create fake data, make a test database, use __*unittest*__, etc.) are all very important. But if you're testing a snippet in a piece of documentation...that might be more scaffolding than you need.

We can keep it simpler. You've met __*os.environ*__, but have you met __*os.system*__?

In [ ]:
import os
   
os.system("dropdb hackbright")
os.system("createdb hackbright")
os.system("psql hackbright < hackbright.sql")

You can run command line programs from inside Python!

Now, if we show sample output that uses the __*hackbright*__ database, as long as you have the right SQL file to run, you can test that output.

(If this code were part of an RST doctest, you'd also have to import __*sys*__ and append the directory where *hackbright.sql* lives to your path. It will probably be your current working directory. For some reason, inside these tests, the files in your current directory can't be found otherwise.)


*__Note:__ Use __os.system__ with caution. This is the kind of code that can result in command line injection if it were to end up in production.*

Namespace Under Test in RST
--------------------------

Inside your Python console and when you run a Python program from the command line, you can reasonably expect your `__name__` to be `"__main__"`. It is here:

In [5]:
print(__name__)

__main__


In [6]:
def display_fruits(fruits):
    """Print a given list of fruits.
    
    >>> display_fruits(["apple", "berry", "cherry"])
    apple
    berry
    cherry
    >>> print(__name__)
    ??? Hark, a doctest failure!
    """
    
    for fruit in fruits:
        print(f"{fruit}")
        
berries = ["apple", "berry", "cherry"]    # Are cherries actually berries?
display_fruits(berries)

apple
berry
cherry


Even in doctests, `__name__` should be `"__main__"`. Proof:

In [7]:
import doctest
doctest.testmod()

**********************************************************************
File "__main__", line 8, in __main__.display_fruits
Failed example:
    print(__name__)
Expected:
    ??? Hark, a doctest failure!
Got:
    __main__
**********************************************************************
1 items had failures:
   1 of   2 in __main__.display_fruits
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=2)

(Also, if you've ever wondered what __*doctest.testmod()*__ gives you back, it's a __*TestResults*__ instance.)

Unfortunately, however, the namespace inside the doctests in our RST files is *not* `__main__`!

Can't really mimic this well here, but here's a failure message you might see:

``` python
**********************************************************************
File "index.rst", line 512, in default
Failed example:
    print __name__
Expected nothing
Got:
    __builtin__
**********************************************************************
```


This makes actions like `app = Flask(__name__)` inside RST doctests a challenge.

(And we don't want to be doing that anyway. Better to make sure that instance can be correctly instantiated in the code we're testing by importing it from our Python file!)
 
The simplest solution here was to change our code slightly. Sometimes, we 
instantiate `app` in a statement like this:

``` python
if __name__ == "__main__":
    app = Flask(__name__)
    
    # ... 
```

Best to just instantiate app at the top of the file, so it can be imported later. You can see this at work in the project-tracker-py exercise.